<a href="https://colab.research.google.com/github/daveAnalyst/zindi-mhp-energy-prediction-2025/blob/master/zindi_mhp_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zindi Micro-Hydropower Energy Load Prediction

This notebook aims to predict daily energy consumption (kWh) per data user for Micro-Hydropower Plants (MHPs) in Kalam, Pakistan. We will use MHP sensor data and climate indicators to build a predictive model.

**Objective:** Forecast total daily kWh per user for one month into the future.
**Metric:** Root Mean Squared Error (RMSE).

In [2]:
# Cell 1A: Setup and Git Sync (Corrected Data Setup)

import os
import time
from google.colab import drive
import zipfile # Make sure zipfile is imported
import shutil # Make sure shutil is imported

print("--- Starting Day 3 Setup ---")
setup_start_time = time.time()

# 1. Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Drive mounted.")

# 2. Define Repo Name and Path
repo_name = "zindi-mhp-energy-prediction-2025"
base_content_path = '/content'
repo_path = os.path.join(base_content_path, repo_name)
print(f"Target repository path: {repo_path}")

# 3. Clone or Change Directory, Configure Git User
print("\nCloning repository if needed...")
if not os.path.exists(repo_path):
    # --- IMPORTANT: Replace with your NEW PAT ---
    PAT = "ghp_YOUR_NEW_PAT_HERE" # Replace with the PAT you generated today
    # ---------------------------------------
    clone_url = f"https://{PAT}@github.com/daveAnalyst/{repo_name}.git"
    !git clone {clone_url}
    if not os.path.exists(repo_path):
         print("--- ERROR: Clone failed! Cannot proceed. ---")
         raise RuntimeError("Git clone failed")
    print("Repository cloned.")
    os.chdir(repo_path)
else:
    print("Repository already exists.")
    if os.path.basename(os.getcwd()) != repo_name:
        os.chdir(repo_path)
print(f"Current directory: {os.getcwd()}")

print("\nConfiguring Git user...")
GIT_EMAIL = "your_github_email@example.com" #<-- YOUR GITHUB EMAIL
GIT_NAME = "Your GitHub Name" #<-- YOUR GITHUB NAME/USERNAME
!git config --global user.email "{GIT_EMAIL}"
!git config --global user.name "{GIT_NAME}"
print("Git user configured.")

# 4. Pull Latest Changes from GitHub
print("\nPulling latest changes from origin/master...")
!git pull origin master
print("Pull complete.")

# 5. Install Required Packages
print("\nInstalling/Updating required packages...")
!pip install optuna lightgbm==4.1.0 openpyxl pandas numpy scikit-learn --quiet
print("Packages installed/updated.")

# 6. Unzip/Copy Data from Drive to Colab Runtime (Corrected Logic)
DRIVE_DATA_PATH = '/content/drive/MyDrive/Zindi MHP Challenge/data' # Adjust if needed
COLAB_DATA_PATH = '/content/data'
DATA_DIR = COLAB_DATA_PATH

print(f"\nSetting up data in '{DATA_DIR}' from '{DRIVE_DATA_PATH}'...")
os.makedirs(DATA_DIR, exist_ok=True)

# --- Unzip Data.csv ---
print("  Unzipping Data.zip...")
try:
    with zipfile.ZipFile(os.path.join(DRIVE_DATA_PATH, 'Data.zip'), 'r') as z: z.extractall(DATA_DIR)
    print("    Data.zip unzipped.")
except FileNotFoundError: print(f"    ERROR: Data.zip not found at {os.path.join(DRIVE_DATA_PATH, 'Data.zip')}")
except Exception as e: print(f"    ERROR unzipping Data.zip: {e}")

# --- Unzip Climate Data.xlsx --- <<< CORRECTION HERE >>>
print("  Unzipping Climate Data.zip...")
climate_zip_path = os.path.join(DRIVE_DATA_PATH, 'Climate Data.zip')
excel_final_name = 'Kalam Climate Data.xlsx' # The target filename
excel_path_in_colab = os.path.join(DATA_DIR, excel_final_name)

try:
    with zipfile.ZipFile(climate_zip_path, 'r') as zip_ref:
        # Extract all contents first (might create subfolder)
        zip_ref.extractall(DATA_DIR)
        print(f"    Climate Data.zip extracted to {DATA_DIR}. Checking contents...")
        # Try to find the Excel file, potentially in a subfolder named like the zip
        extracted_excel_path = None
        possible_subfolder_name = 'Climate Data' # Common pattern
        path_in_subfolder = os.path.join(DATA_DIR, possible_subfolder_name, excel_final_name)
        path_direct = os.path.join(DATA_DIR, excel_final_name)

        if os.path.exists(path_in_subfolder):
             print(f"    Found Excel file in subfolder: {path_in_subfolder}")
             # Move it out of the subfolder
             print(f"    Moving Excel file to {excel_path_in_colab}")
             shutil.move(path_in_subfolder, excel_path_in_colab)
             # Optionally remove the now empty subfolder and __MACOSX if it exists
             if os.path.exists(os.path.join(DATA_DIR, possible_subfolder_name)): shutil.rmtree(os.path.join(DATA_DIR, possible_subfolder_name))
             if os.path.exists(os.path.join(DATA_DIR, '__MACOSX')): shutil.rmtree(os.path.join(DATA_DIR, '__MACOSX'))
             extracted_excel_path = excel_path_in_colab
        elif os.path.exists(path_direct):
             print(f"    Found Excel file directly: {path_direct}")
             extracted_excel_path = path_direct
        else:
             print(f"    ERROR: Cannot find '{excel_final_name}' after extracting zip.")
             print(f"    Contents of {DATA_DIR}:")
             !ls -R "{DATA_DIR}" # Show contents to help debug

    if extracted_excel_path and os.path.exists(extracted_excel_path):
        print(f"    Climate data Excel file is ready at: {extracted_excel_path}")
    else:
        print(f"    ERROR: Problem setting up climate data Excel file.")

except FileNotFoundError: print(f"    ERROR: Climate Data.zip not found at {climate_zip_path}")
except Exception as e: print(f"    ERROR unzipping Climate Data.zip: {e}")


# --- Copy SampleSubmission.csv ---
print("  Copying SampleSubmission.csv...")
try:
    shutil.copy(os.path.join(DRIVE_DATA_PATH, 'SampleSubmission.csv'), DATA_DIR)
    print("    SampleSubmission.csv copied.")
except FileNotFoundError: print(f"    ERROR: SampleSubmission.csv not found at {os.path.join(DRIVE_DATA_PATH, 'SampleSubmission.csv')}")
except Exception as e: print(f"    ERROR copying SampleSubmission.csv: {e}")


print("\nFinal check of Colab data directory:")
!ls -lh "{DATA_DIR}"

setup_time = time.time() - setup_start_time
print(f"\n--- Setup finished in {setup_time:.1f}s ---")
print(f"\nIMPORTANT: Ensure DATA_DIR for subsequent cells is: {DATA_DIR}")

--- Starting Day 3 Setup ---
Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted.
Target repository path: /content/zindi-mhp-energy-prediction-2025

Cloning repository if needed...
Repository already exists.
Current directory: /content/zindi-mhp-energy-prediction-2025

Configuring Git user...
Git user configured.

Pulling latest changes from origin/master...
From https://github.com/daveAnalyst/zindi-mhp-energy-prediction-2025
 * branch            master     -> FETCH_HEAD
Already up to date.
Pull complete.

Installing/Updating required packages...
Packages installed/updated.

Setting up data in '/content/data' from '/content/drive/MyDrive/Zindi MHP Challenge/data'...
  Unzipping Data.zip...
    Data.zip unzipped.
  Unzipping Climate Data.zip...
    Climate Data.zip extracted to /content/data. Checking contents...
    Found Excel file in subfolder: /content/data/Climate Dat

In [3]:
# Basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math # For sqrt
import os

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Settings
SEED = 42
np.random.seed(SEED)
DATA_DIR = 'data' # Set the path to your data directory

# Ensure plots are displayed inline and set a style
%matplotlib inline
# Use a style that's likely available - adjust if needed
plt.style.use('seaborn-v0_8-darkgrid')
# If the above fails, try: plt.style.use('seaborn-darkgrid') or plt.style.use('ggplot')

print("Libraries imported and settings configured.")
print(f"Data directory set to: {DATA_DIR}")
print(f"Available plotting styles: {plt.style.available}") # Optional: see available styles

Libraries imported and settings configured.
Data directory set to: data
Available plotting styles: ['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [4]:
import os

# ---- IMPORTANT: SET THIS CORRECTLY ----
# Option 1: If you unzipped/copied data to Colab runtime storage
DATA_DIR = '/content/data'
# Option 2: If you are reading directly from Google Drive (potentially slower)
# DATA_DIR = '/content/drive/MyDrive/Zindi MHP Challenge/data' # Adjust path if needed
# --------------------------------------

print(f"Using DATA_DIR: {DATA_DIR}")

# Check 1: Does DATA_DIR exist?
print(f"\nChecking existence of DATA_DIR...")
if not os.path.exists(DATA_DIR):
    print(f"--- ERROR ---: Directory '{DATA_DIR}' does NOT exist!")
    print("Action: Ensure you have run the Drive mount AND the data unzipping/copying cell correctly in this session.")
else:
    print(f"Directory '{DATA_DIR}' exists.")

    # Check 2: List files in DATA_DIR (Check names and capitalization!)
    print(f"\nFiles present in '{DATA_DIR}':")
    !ls -lh "{DATA_DIR}" # Use shell command for clear listing

    # Check 3: Verify specific expected files
    expected_files = ['Data.csv', 'Kalam Climate Data.xlsx', 'SampleSubmission.csv']
    print("\nChecking for expected files:")
    all_found = True
    for f_name in expected_files:
        f_path = os.path.join(DATA_DIR, f_name)
        if os.path.exists(f_path):
            print(f"  [ OK ] Found: {f_name}")
        else:
            print(f"  [FAIL] MISSING: {f_name} at path {f_path}")
            all_found = False
    if not all_found:
        print("--- ERROR ---: One or more required data files are missing from DATA_DIR.")
        print("Action: Re-run the data unzipping/copying cell, checking its output for errors.")
    else:
        print("All expected files seem to be present.")

# Check 4: Ensure openpyxl is installed (relevant for Excel)
print("\nChecking for openpyxl installation...")
try:
    import openpyxl
    print("  [ OK ] openpyxl is installed.")
except ImportError:
    print("  [FAIL] openpyxl is NOT installed.")
    print("Action: Run '!pip install openpyxl' in a cell.")

Using DATA_DIR: /content/data

Checking existence of DATA_DIR...
Directory '/content/data' exists.

Files present in '/content/data':
total 2.8G
-rw-r--r-- 1 root root 2.8G Apr  6 12:25  Data.csv
-rw-r--r-- 1 root root 735K Apr  6 12:25 'Kalam Climate Data.xlsx'
-rw------- 1 root root 260K Apr  6 12:25  SampleSubmission.csv

Checking for expected files:
  [ OK ] Found: Data.csv
  [ OK ] Found: Kalam Climate Data.xlsx
  [ OK ] Found: SampleSubmission.csv
All expected files seem to be present.

Checking for openpyxl installation...
  [ OK ] openpyxl is installed.


In [5]:
import pandas as pd
import os

# --- Use the SAME DATA_DIR as verified in Step 1 ---
DATA_DIR = '/content/data' # Or your Drive path
# ----------------------------------------------------

print("--- Verifying Column Names ---")

# Check CSV Columns
try:
    print("\nChecking Data.csv headers...")
    csv_path = os.path.join(DATA_DIR, 'Data.csv')
    # Load only 5 rows, use engine='python' if C engine failed before
    csv_head = pd.read_csv(csv_path, nrows=5, engine='python')
    print(f"Columns found in Data.csv: {csv_head.columns.tolist()}")
    # --->>> CONFIRM: Are 'date_time' and 'Source' exactly as expected? <<<---
except Exception as e:
    print(f"ERROR reading Data.csv header: {e}")

# Check Excel Columns
try:
    print("\nChecking Kalam Climate Data.xlsx headers...")
    excel_path = os.path.join(DATA_DIR, 'Kalam Climate Data.xlsx')
    excel_head = pd.read_excel(excel_path, nrows=5) # Reads first sheet by default
    print(f"Columns found in Kalam Climate Data.xlsx: {excel_head.columns.tolist()}")
    # --->>> CONFIRM: Is 'Date_Time' exactly as expected? <<<---
    # Let's also see the format of the date column
    date_col_name = 'Date_Time' # Use the actual column name found
    if date_col_name in excel_head.columns:
         print(f"Sample values in Excel column '{date_col_name}':")
         print(excel_head[date_col_name].head())
    else:
         print(f"Warning: Column '{date_col_name}' not found in Excel header check.")

except Exception as e:
    print(f"ERROR reading Kalam Climate Data.xlsx header: {e}")

--- Verifying Column Names ---

Checking Data.csv headers...
Columns found in Data.csv: ['date_time', 'v_red', 'current', 'power_factor', 'kwh', 'Source', 'v_blue', 'v_yellow', 'consumer_device_9', 'consumer_device_x']

Checking Kalam Climate Data.xlsx headers...
Columns found in Kalam Climate Data.xlsx: ['Date Time', 'Temperature (°C)', 'Dewpoint Temperature (°C)', 'U Wind Component (m/s)', 'V Wind Component (m/s)', 'Total Precipitation (mm)', 'Snowfall (mm)', 'Snow Cover (%)']


## 2. Load Data

Load the datasets:
1.  **MHP Data:** `Data.csv` contains the 5-minute interval sensor readings (voltage, current, kWh, etc.). We anticipate the timestamp column is named `date_time`.
2.  **Climate Data:** `Kalam Climate Data.xlsx` contains climate indicators (temperature, precipitation, etc.). This is an Excel file.
3.  **Sample Submission:** `SampleSubmission.csv` defines the required prediction format and IDs for the test set.

*Note: Reading Excel files requires the `openpyxl` library. Install it if needed (`pip install openpyxl`). Reading the MHP CSV might require `engine='python'` if the default C engine fails.*

In [6]:
# Cell 4: Load Data (Revised with Correct Column Names)

import pandas as pd
import numpy as np # Make sure numpy is imported
import os
import time

# --- Use the SAME DATA_DIR as verified previously ---
# Ensure this is set correctly for your Colab environment
DATA_DIR = '/content/data' # Or your Drive path: '/content/drive/MyDrive/Zindi MHP Challenge/data'
# ----------------------------------------------------

print("Loading data with verified column names...")
start_time = time.time()

try:
    # --- Load MHP Data (CSV) ---
    mhp_filename = 'Data.csv'
    mhp_path = os.path.join(DATA_DIR, mhp_filename)
    print(f"\n[{time.time() - start_time:.1f}s] Attempting to load MHP structure from: {mhp_path}")

    # Columns verified as correct in Data.csv
    mhp_cols_to_load = ['date_time', 'Source', 'kwh']
    print(f"Loading only columns: {mhp_cols_to_load}")

    mhp_data_raw = pd.read_csv(
        mhp_path,
        usecols=mhp_cols_to_load,
        #engine='python'  Keep if C engine failed
    )
    print(f"[{time.time() - start_time:.1f}s] Structure loaded for {mhp_filename}. Size: {mhp_data_raw.shape}")

    # Convert date column AFTER loading
    print(f"[{time.time() - start_time:.1f}s] Converting MHP date column ('date_time')...")
    mhp_data_raw['timestamp'] = pd.to_datetime(mhp_data_raw['date_time'], errors='coerce')
    if mhp_data_raw['timestamp'].isnull().any():
        print("Warning: Some MHP date conversions failed (resulted in NaT).")
    mhp_data_raw.drop(columns=['date_time'], inplace=True) # Drop original string column
    print(f"[{time.time() - start_time:.1f}s] MHP date converted.")

    # Rename user ID column (verified as 'Source')
    mhp_data_raw.rename(columns={'Source': 'user_id'}, inplace=True)
    print(f"[{time.time() - start_time:.1f}s] Loaded and initially processed: {mhp_filename}")


    # --- Load Climate Data (Excel) ---
    climate_filename = 'Kalam Climate Data.xlsx'
    climate_path = os.path.join(DATA_DIR, climate_filename)
    print(f"\n[{time.time() - start_time:.1f}s] Attempting to load Excel structure from: {climate_path}")
    # Load full Excel structure first, do not parse dates yet
    climate_data_raw = pd.read_excel(climate_path)
    print(f"[{time.time() - start_time:.1f}s] Structure loaded for {climate_filename}. Size: {climate_data_raw.shape}")
    print(f"Original Excel columns: {climate_data_raw.columns.tolist()}") # Show original names

    # --- Rename Excel Columns to Simpler Names ---
    # Use the EXACT names found in the verification step
    print(f"[{time.time() - start_time:.1f}s] Renaming Excel columns...")
    climate_rename_map = {
        'Date Time': 'date_time_excel', # Temporary name for date
        'Temperature (°C)': 'temperature',
        'Dewpoint Temperature (°C)': 'dew_point',
        'U Wind Component (m/s)': 'u_wind',
        'V Wind Component (m/s)': 'v_wind',
        'Total Precipitation (mm)': 'precipitation'
        # Add other columns here if needed later, e.g., 'Snowfall (mm)': 'snowfall'
    }
    # Check if all expected columns exist before renaming
    missing_cols = [col for col in climate_rename_map.keys() if col not in climate_data_raw.columns]
    if missing_cols:
         raise KeyError(f"Missing expected columns in Excel file: {missing_cols}")

    climate_data_raw.rename(columns=climate_rename_map, inplace=True)
    print(f"Columns after renaming: {climate_data_raw.columns.tolist()}")

    # --- Calculate Wind Speed Magnitude ---
    print(f"[{time.time() - start_time:.1f}s] Calculating wind speed magnitude...")
    climate_data_raw['wind_speed'] = np.sqrt(climate_data_raw['u_wind']**2 + climate_data_raw['v_wind']**2)
    # Drop original U/V components if no longer needed
    # climate_data_raw.drop(columns=['u_wind', 'v_wind'], inplace=True)
    print(f"Wind speed calculated.")

    # --- Convert Date Column AFTER Loading and Renaming ---
    print(f"[{time.time() - start_time:.1f}s] Converting Excel date column ('date_time_excel')...")
    climate_data_raw['timestamp'] = pd.to_datetime(climate_data_raw['date_time_excel'], errors='coerce')
    if climate_data_raw['timestamp'].isnull().any():
        print("Warning: Some Excel date conversions failed (resulted in NaT).")
    # Drop original date column AFTER successful conversion
    climate_data_raw.drop(columns=['date_time_excel'], inplace=True)
    print(f"[{time.time() - start_time:.1f}s] Excel date converted.")
    print(f"[{time.time() - start_time:.1f}s] Loaded and initially processed: {climate_filename}")


    # --- Load Sample Submission (CSV) ---
    sample_sub_filename = 'SampleSubmission.csv'
    sample_sub_path = os.path.join(DATA_DIR, sample_sub_filename)
    print(f"\n[{time.time() - start_time:.1f}s] Attempting to load sample submission from: {sample_sub_path}")
    sample_sub = pd.read_csv(sample_sub_path)
    print(f"[{time.time() - start_time:.1f}s] Successfully loaded: {sample_sub_filename}")

    total_time = time.time() - start_time
    print(f"\n[{total_time:.1f}s] All data loaded successfully.")

# --- Keep the SAME except blocks as before ---
except FileNotFoundError as e:
    print(f"\n--- ERROR: File Not Found ---")
    print(f"{e}")
    print(f"Please ensure files are directly in the '{DATA_DIR}' directory confirmed in Step 1.")
    raise
except ImportError as e:
     print(f"\n--- ERROR: Missing Library ---")
     print(f"{e}")
     print("Failed loading Excel file. Ensure openpyxl is installed (`!pip install openpyxl`).")
     raise
except ValueError as e:
    print(f"\n--- ERROR: Value Error during loading/parsing ---")
    print(f"{e}")
    print("This might be due to an incorrect column name during processing. Verify names from Step 2.")
    raise
except KeyError as e:
    print(f"\n--- ERROR: Key Error ---")
    print(f"{e}")
    print("This usually means a specified column name was not found during renaming or processing. Verify names from Step 2.")
    raise
except Exception as e:
     print(f"\n--- ERROR: An unexpected error occurred during data loading ---")
     print(f"{e}")
     raise

# --- Display Info (Optional but recommended) ---
# Select only the columns we will actually use going forward to keep info clean
mhp_final_cols = ['timestamp', 'user_id', 'kwh']
climate_final_cols = ['timestamp', 'temperature', 'dew_point', 'wind_speed', 'precipitation'] # Add others like 'snowfall' if needed

if 'mhp_data_raw' in locals():
    print("\n--- MHP Data Info (Post-Processing) ---")
    mhp_data_raw[mhp_final_cols].info() # Show info only for relevant columns
if 'climate_data_raw' in locals():
    print("\n--- Climate Data Info (Post-Processing) ---")
    climate_data_raw[climate_final_cols].info() # Show info only for relevant columns
if 'sample_sub' in locals():
    print("\n--- Sample Submission Head ---")
    print(sample_sub.head())

Loading data with verified column names...

[0.0s] Attempting to load MHP structure from: /content/data/Data.csv
Loading only columns: ['date_time', 'Source', 'kwh']
[60.6s] Structure loaded for Data.csv. Size: (39147559, 3)
[60.6s] Converting MHP date column ('date_time')...
[77.1s] MHP date converted.
[77.1s] Loaded and initially processed: Data.csv

[77.1s] Attempting to load Excel structure from: /content/data/Kalam Climate Data.xlsx
[80.1s] Structure loaded for Kalam Climate Data.xlsx. Size: (12228, 8)
Original Excel columns: ['Date Time', 'Temperature (°C)', 'Dewpoint Temperature (°C)', 'U Wind Component (m/s)', 'V Wind Component (m/s)', 'Total Precipitation (mm)', 'Snowfall (mm)', 'Snow Cover (%)']
[80.1s] Renaming Excel columns...
Columns after renaming: ['date_time_excel', 'temperature', 'dew_point', 'u_wind', 'v_wind', 'precipitation', 'Snowfall (mm)', 'Snow Cover (%)']
[80.1s] Calculating wind speed magnitude...
Wind speed calculated.
[80.1s] Converting Excel date column ('d

## 3. Data Preprocessing and Aggregation

The MHP data is recorded at 5-minute intervals, but the prediction target is daily kWh per user. Climate data might also be at a finer granularity than daily.

We need to:
1.  Aggregate the MHP `kwh` readings to get the total daily sum for each `user_id`.
2.  Aggregate the climate data to daily statistics (e.g., mean temperature, total precipitation).
3.  Merge the aggregated daily MHP data with the aggregated daily climate data.
4.  Handle any missing values that might arise from the merge or exist in the original data.

In [7]:
# Cell 6: Data Preprocessing and Aggregation (Corrected Syntax)

print("Aggregating data to daily level...")
agg_start_time = time.time() # Start timer for aggregation

# --- MHP Data Aggregation ---
# Ensure required columns exist from Cell 4 processing
if 'timestamp' not in mhp_data_raw.columns: raise KeyError("Column 'timestamp' not found in mhp_data_raw.")
if 'user_id' not in mhp_data_raw.columns: raise KeyError("Column 'user_id' not found in mhp_data_raw.")
if 'kwh' not in mhp_data_raw.columns: raise KeyError("Column 'kwh' not found in mhp_data_raw.")

mhp_data = mhp_data_raw.copy()
# Extract date from the timestamp
mhp_data['date'] = mhp_data['timestamp'].dt.date

# Group by the user ID (already named 'user_id') and the date, then sum the kWh for that day
print(f"\n[{time.time() - agg_start_time:.1f}s] Grouping MHP data by user_id and date...")
# This can be memory intensive for 40M rows. Monitor Colab RAM.
daily_kwh = mhp_data.groupby(['user_id', 'date'])['kwh'].sum().reset_index()
print(f"[{time.time() - agg_start_time:.1f}s] MHP aggregation complete.")
daily_kwh.rename(columns={'kwh': 'daily_kwh'}, inplace=True)
# Convert date back to datetime object for merging and feature engineering
daily_kwh['date'] = pd.to_datetime(daily_kwh['date'])
print(f"Aggregated MHP data shape: {daily_kwh.shape}")
print("Aggregated Daily kWh per User Head:")
print(daily_kwh.head())


# --- Climate Data Aggregation (Using simplified names from Cell 4) ---
# --- Climate Data Aggregation (Using simplified names from Cell 4 - CORRECTED .agg() call) ---
# Ensure required columns exist
if 'timestamp' not in climate_data_raw.columns: raise KeyError("Column 'timestamp' not found in climate_data_raw.")

climate_data = climate_data_raw.copy()
# Extract date from the timestamp
climate_data['date'] = climate_data['timestamp'].dt.date

# Define the columns to aggregate and the aggregation function
climate_agg_dict = {
    'temperature': 'mean',
    'dew_point': 'mean',
    'wind_speed': 'mean',
    'precipitation': 'sum'
    # Add 'u_wind':'mean', 'v_wind':'mean' if you want to keep daily components
}
# Check if needed columns exist in climate_data before aggregation
required_climate_cols = list(climate_agg_dict.keys())
missing_climate_cols = [col for col in required_climate_cols if col not in climate_data.columns]
if missing_climate_cols:
    raise KeyError(f"Missing required climate columns for aggregation: {missing_climate_cols}. Check Cell 4 processing.")

print(f"\n[{time.time() - agg_start_time:.1f}s] Aggregating climate data...")

# --- CORRECTED .agg() call ---
# Pass the dictionary directly as the argument
daily_climate = climate_data.groupby('date').agg(climate_agg_dict).reset_index()

# Rename columns to include the aggregation type (e.g., temp_mean) - important!
# Create the names expected by later cells (e.g., 'temp_mean', not 'temperature_mean')
rename_map = {
    'temperature': 'temp_mean',
    'dew_point': 'dew_point_mean',
    'wind_speed': 'wind_speed_mean',
    'precipitation': 'precip_sum'
}
# Only rename columns that were actually aggregated
rename_map = {k: v for k, v in rename_map.items() if k in daily_climate.columns}
daily_climate.rename(columns=rename_map, inplace=True)
# --- End of Correction ---


print(f"[{time.time() - agg_start_time:.1f}s] Climate aggregation complete.")

# Convert date back to datetime object
daily_climate['date'] = pd.to_datetime(daily_climate['date'])
print(f"Aggregated climate data shape: {daily_climate.shape}")
print("Columns after renaming: {daily_climate.columns.tolist()}") # Verify names
print("Aggregated Daily Climate Head:")
print(daily_climate.head())



# --- Merge Aggregated Data ---
print(f"\n[{time.time() - agg_start_time:.1f}s] Merging aggregated daily MHP and climate data...")
df_train_full = pd.merge(daily_kwh, daily_climate, on='date', how='left')
print(f"[{time.time() - agg_start_time:.1f}s] Merge complete. Initial merged shape: {df_train_full.shape}")

# --- Handle Missing Values ---
print(f"\n[{time.time() - agg_start_time:.1f}s] Checking NaNs before fill:\n{df_train_full.isnull().sum()}")
# Sort before filling for consistent ffill/bfill
df_train_full.sort_values(by=['user_id', 'date'], inplace=True)
df_train_full.fillna(method='ffill', inplace=True)
df_train_full.fillna(method='bfill', inplace=True)
nan_count_after = df_train_full.isnull().sum().sum()
print(f"\n[{time.time() - agg_start_time:.1f}s] FillNA complete. NaNs remaining: {nan_count_after}")

agg_total_time = time.time() - agg_start_time
print(f"\n[{agg_total_time:.1f}s] Cell 6 processing finished.")
print("\n--- Merged & Cleaned Training Data Head ---")
print(df_train_full.head())

Aggregating data to daily level...

[17.2s] Grouping MHP data by user_id and date...
[31.6s] MHP aggregation complete.
Aggregated MHP data shape: (136409, 3)
Aggregated Daily kWh per User Head:
                          user_id       date  daily_kwh
0  consumer_device_10_data_user_1 2024-07-22   0.024330
1  consumer_device_10_data_user_1 2024-07-23   0.103560
2  consumer_device_10_data_user_1 2024-07-24   0.137543
3  consumer_device_10_data_user_1 2024-07-25   0.121011
4  consumer_device_10_data_user_1 2024-07-26   0.000000

[31.6s] Aggregating climate data...
[31.7s] Climate aggregation complete.
Aggregated climate data shape: (511, 5)
Columns after renaming: {daily_climate.columns.tolist()}
Aggregated Daily Climate Head:
        date  temp_mean  dew_point_mean  wind_speed_mean  precip_sum
0 2023-06-03   1.860280       -3.348664         0.680343    0.004557
1 2023-06-04   3.992740       -1.905203         0.672389    0.024096
2 2023-06-05   4.794523       -3.781657         0.679359    

<ipython-input-7-1e8fee3ca695>:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_train_full.fillna(method='ffill', inplace=True)
<ipython-input-7-1e8fee3ca695>:93: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_train_full.fillna(method='bfill', inplace=True)


## 4. Feature Engineering

Create features based on the date and potentially other aspects of the data. For this baseline, we will focus on date-based features. More advanced features (lags, rolling windows, user-specific stats) can be added later for improvement.

In [13]:
# Cell 8: Feature Engineering (Date + Lag + UserID - NO Rolling)

import time
import pandas as pd # Ensure pandas is imported
import numpy as np  # Ensure numpy is imported

feature_eng_start_time = time.time() # Start timer

# --- Check for input DataFrame ---
if 'df_train_full' not in locals():
    raise NameError("df_train_full not found. Ensure Cell 6 ran successfully.")

# --- 0. Convert user_id to Category dtype ---
print("--- Step 0: Converting user_id to Category ---")
if 'user_id' in df_train_full.columns:
    print(f"Original user_id dtype: {df_train_full['user_id'].dtype}")
    df_train_full['user_id'] = df_train_full['user_id'].astype('category')
    print(f"Converted user_id dtype: {df_train_full['user_id'].dtype}")
else:
    raise KeyError("Column 'user_id' not found in df_train_full.")


# --- 1. Create Lag Features ---
print("\n--- Step 1: Creating Lag Features ---")
print("Sorting data by user_id and date...")
df_train_full = df_train_full.sort_values(by=['user_id', 'date'])

lags_to_create = [1, 2, 3, 7, 14, 28] # Lag periods in days
lag_col_name_base = 'daily_kwh_lag_'
lag_cols_created = [] # Keep track of new lag column names

target_col_for_lags = 'daily_kwh'
print(f"Calculating lags for column: '{target_col_for_lags}' using periods: {lags_to_create}")
for lag in lags_to_create:
    col_name = f"{lag_col_name_base}{lag}"
    # print(f"  Creating lag: {lag} days -> {col_name}") # Reduce verbosity
    # Use observed=True for category dtype groupbys
    df_train_full[col_name] = df_train_full.groupby('user_id', observed=True)[target_col_for_lags].shift(lag)
    lag_cols_created.append(col_name)
print(f"[{time.time() - feature_eng_start_time:.1f}s] Created lag features: {lag_cols_created}")


# --- 2. Create Rolling Window Features (COMMENTED OUT FOR NOW) ---
# print("\n--- Step 2: Creating Rolling Window Features (SKIPPED) ---")
# rolling_cols_created = [] # Ensure list exists even if empty
# (Keep the rolling feature code commented out or deleted)


# --- 3. Create Date Features ---
print("\n--- Step 3: Creating Date Features ---")
def create_date_features(df, date_col='date'):
    """Creates time series features from a date column."""
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day
    df['dayofweek'] = df[date_col].dt.dayofweek
    df['dayofyear'] = df[date_col].dt.dayofyear
    df['weekofyear'] = df[date_col].dt.isocalendar().week.astype(int)
    df['quarter'] = df[date_col].dt.quarter
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    return df

df_train_full = create_date_features(df_train_full, 'date')
print(f"[{time.time() - feature_eng_start_time:.1f}s] Date features applied.")


# --- 4. Define Final Feature List ---
print("\n--- Step 4: Defining Final Feature List (NO Rolling Features) ---")
base_features = [
    'year', 'month', 'day', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter', 'is_weekend',
    'temp_mean', 'dew_point_mean', 'wind_speed_mean', 'precip_sum'
]
features = list(base_features) # Start with base

# Add user_id feature
features.append('user_id')
print("Added 'user_id' to features list.")

# Add lag features
if 'lag_cols_created' in locals() and isinstance(lag_cols_created, list):
    print(f"Adding {len(lag_cols_created)} lag features.")
    features.extend(lag_cols_created)
else: print("WARNING: Lag features ('lag_cols_created') not found or not a list.")

# --- ROLLING FEATURES EXCLUDED ---
print("--- Rolling window features EXCLUDED for this experiment ---")
# if 'rolling_cols_created' in locals() and isinstance(rolling_cols_created, list):
#    features.extend(rolling_cols_created) # Ensure this is commented out/deleted

target = 'daily_kwh'
print(f"\nFinal list of features DEFINED ({len(features)}): {features}") # Check this output


# --- 5. Check Feature Data Types and Handle NaNs ---
print("\n--- Step 5: Checking Data Types & Handling NaNs ---")
missing_features = [f for f in features if f not in df_train_full.columns]
if missing_features: raise ValueError(f"Features defined but missing: {missing_features}")

print("Checking for non-numeric feature columns (excluding category)...")
for col in features:
    if col != 'user_id' and not pd.api.types.is_numeric_dtype(df_train_full[col]):
        print(f"  Warning: Feature '{col}' not numeric ({df_train_full[col].dtype}). Converting.")
        df_train_full[col] = pd.to_numeric(df_train_full[col], errors='coerce')
        if df_train_full[col].isnull().any():
             print(f"    NaNs from conversion in {col}, filling with 0.")
             df_train_full[col].fillna(0, inplace=True)

print(f"\nHandling remaining NaNs in features (filling with 0)...")
numeric_features_for_nan_fill = [f for f in features if f != 'user_id']
nan_counts_before_fill = df_train_full[numeric_features_for_nan_fill].isnull().sum()
print("NaN counts BEFORE fill (numeric features only):")
print(nan_counts_before_fill[nan_counts_before_fill > 0])
df_train_full[numeric_features_for_nan_fill] = df_train_full[numeric_features_for_nan_fill].fillna(0)
nan_counts_after_fill = df_train_full[numeric_features_for_nan_fill].isnull().sum().sum()
if nan_counts_after_fill == 0: print(f"\n[ OK ] NaNs handled. Total remaining (numeric): {nan_counts_after_fill}")
else: print(f"\n[ WARNING ] NaNs remain after fill (numeric): {nan_counts_after_fill}.")

feature_eng_total_time = time.time() - feature_eng_start_time
print(f"\n[{feature_eng_total_time:.1f}s] Cell 8 processing finished.")

print("\n--- Training Data with Features Head (Sample) ---")
# Display head including user_id and some lags
cols_to_show = (['user_id', 'date', target] + base_features[:2] + ['user_id'] +
                lag_cols_created[:2] ) # Removed rolling cols from print
print(df_train_full[cols_to_show].head())

--- Step 0: Converting user_id to Category ---
Original user_id dtype: category
Converted user_id dtype: category

--- Step 1: Creating Lag Features ---
Sorting data by user_id and date...
Calculating lags for column: 'daily_kwh' using periods: [1, 2, 3, 7, 14, 28]
[0.1s] Created lag features: ['daily_kwh_lag_1', 'daily_kwh_lag_2', 'daily_kwh_lag_3', 'daily_kwh_lag_7', 'daily_kwh_lag_14', 'daily_kwh_lag_28']

--- Step 3: Creating Date Features ---
[0.1s] Date features applied.

--- Step 4: Defining Final Feature List (NO Rolling Features) ---
Added 'user_id' to features list.
Adding 6 lag features.
--- Rolling window features EXCLUDED for this experiment ---

Final list of features DEFINED (19): ['year', 'month', 'day', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter', 'is_weekend', 'temp_mean', 'dew_point_mean', 'wind_speed_mean', 'precip_sum', 'user_id', 'daily_kwh_lag_1', 'daily_kwh_lag_2', 'daily_kwh_lag_3', 'daily_kwh_lag_7', 'daily_kwh_lag_14', 'daily_kwh_lag_28']

--- Step 5: C

## 5. Train/Validation Split (Time-Based)

For time series forecasting, it's crucial to validate the model on data that comes *after* the training data. We will split the data chronologically, using the most recent period for validation. Shuffling should **not** be used.

In [20]:
# Cell 9A: Optuna Hyperparameter Tuning (Corrected for user_id handling)

import optuna
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import math
import numpy as np
import pandas as pd # Ensure pandas is imported
import time

print("--- Starting Optuna Hyperparameter Tuning ---")
optuna_start_time = time.time()

# --- Check for required inputs from Cell 8 ---
if 'df_train_full' not in locals(): raise NameError("df_train_full not found.")
# *** Crucially, use the 'features' list DEFINED AND FINALIZED in Cell 8 ***
if 'features' not in locals(): raise NameError("features list not defined.")
if 'target' not in locals(): raise NameError("target variable name not defined.")
if 'user_id' not in features: raise ValueError("'user_id' MUST be included in the 'features' list defined in Cell 8.")
if 'SEED' not in locals(): SEED = 42; print("Warning: SEED not found, using 42.")

# --- Prepare full dataset for Optuna ---
# Ensure correct sorting and data types FROM Cell 8 output
df_train_full_opt = df_train_full.sort_values(by='date') # Use the dataframe processed by Cell 8

# Ensure user_id is category type *before* splitting
if df_train_full_opt['user_id'].dtype.name != 'category':
    print("Error: user_id dtype is not 'category' after Cell 8. Check Cell 8.")
    # Force conversion again just in case, but Cell 8 should handle it
    df_train_full_opt['user_id'] = df_train_full_opt['user_id'].astype('category')

# Create X and y using the DEFINITIVE features list from Cell 8
X_full_opt = df_train_full_opt[features]
y_full_opt = df_train_full_opt[target]
print(f"Data shape for Optuna: X={X_full_opt.shape}, y={y_full_opt.shape}")
print(f"Features used ({len(features)}): {X_full_opt.columns.tolist()}") # Verify features used
# Verify user_id dtype again
print(f"user_id dtype in X_full_opt: {X_full_opt['user_id'].dtype}") # Should be 'category'


# --- Define Optuna Objective Function ---
def objective(trial):
    # --- Suggest Hyperparameters ---
    params = {
        'objective': 'regression_l1',
        'metric': 'rmse',
        'n_estimators': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 60),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 0.9),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 0.9),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.1, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.1, 10.0, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'seed': SEED,
        'n_jobs': -1,
        'verbose': -1,
        'boosting_type': 'gbdt'
    }

    # --- Run CV with suggested params ---
    N_SPLITS_OPT = 5
    tscv_opt = TimeSeriesSplit(n_splits=N_SPLITS_OPT)
    fold_scores_opt = []
    zero_rmse_folds_opt = []

    # Use the prepared X_full_opt and y_full_opt which have correct dtypes
    for fold, (train_index, val_index) in enumerate(tscv_opt.split(X_full_opt)):
        X_train, X_val = X_full_opt.iloc[train_index], X_full_opt.iloc[val_index]
        y_train, y_val = y_full_opt.iloc[train_index], y_full_opt.iloc[val_index]

        # Initialize model for the fold
        model_fold = lgb.LGBMRegressor(**params)

        # Fit the model - NO 'categorical_feature' argument needed here
        # because X_train['user_id'] already has 'category' dtype
        try:
            model_fold.fit(X_train, y_train,
                           eval_set=[(X_val, y_val)],
                           eval_metric='rmse',
                           callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
                           )
        except Exception as e:
            print(f"!!! Error during model.fit in Trial {trial.number}, Fold {fold+1}: {e}")
            # Optionally return a high error value for this trial
            return float('inf')


        val_preds_fold = model_fold.predict(X_val)
        val_preds_fold = np.maximum(0, val_preds_fold) # Clip negatives
        # Handle potential NaNs in predictions if fit failed partially (unlikely with exception handling)
        val_preds_fold = np.nan_to_num(val_preds_fold, nan=0.0) # Replace NaN preds with 0

        # Ensure y_val has no NaNs for metric calculation
        if y_val.isnull().any():
             print(f"Warning: NaNs found in y_val for Trial {trial.number}, Fold {fold+1}. Check data.")
             # Decide how to handle: skip fold, fill y_val (not recommended), or error
             continue # Skip this fold if target is NaN

        rmse_fold = math.sqrt(mean_squared_error(y_val, val_preds_fold))

        if np.isclose(rmse_fold, 0.0) and y_val.nunique() > 1: # Check if truly zero or just constant target
            print(f"Warning: Fold {fold+1} in Trial {trial.number} has RMSE ~0 but target is not constant.")
            zero_rmse_folds_opt.append(fold)
        elif y_val.nunique() <= 1 and np.isclose(rmse_fold, 0.0) :
             # This is the expected zero fold case
             zero_rmse_folds_opt.append(fold)

        fold_scores_opt.append(rmse_fold)

    # --- Calculate and Return Realistic CV Score ---
    realistic_scores = [score for i, score in enumerate(fold_scores_opt) if i not in zero_rmse_folds_opt]
    if not realistic_scores:
        print(f"Warning: Trial {trial.number} resulted in all zero/invalid RMSE folds.")
        # Check if fold_scores_opt has any non-zero values at all
        if any(not np.isclose(s, 0.0) for s in fold_scores_opt):
             # If there were non-zero scores but they were excluded, maybe use all scores
             print("Using all fold scores as fallback.")
             mean_realistic_cv_rmse = np.mean(fold_scores_opt)
        else:
             # All scores were genuinely zero
             mean_realistic_cv_rmse = 0.0 # Or return float('inf')? Let's return 0 for now.
    else:
        mean_realistic_cv_rmse = np.mean(realistic_scores)

    # print(f"Trial {trial.number} Realistic RMSE: {mean_realistic_cv_rmse:.4f}") # Optional: progress print
    return mean_realistic_cv_rmse

# --- Create and Run Optuna Study ---
N_TRIALS = 20 # Keep at 50 for now
print(f"\nRunning Optuna study with {N_TRIALS} trials...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

# --- Report Best Results ---
print("\n--- Optuna Tuning Summary ---")
print(f"Number of finished trials: {len(study.trials)}")
if study.best_trial: # Check if a best trial exists
    print("Best trial:")
    best_trial = study.best_trial
    print(f"  Value (Mean Realistic CV RMSE): {best_trial.value:.4f}")
    print("  Params: ")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")
    # Store best params for next step
    best_params_from_optuna = best_trial.params
else:
    print("Study finished without finding a best trial (all might have failed).")
    best_params_from_optuna = None # Indicate failure

optuna_time = time.time() - optuna_start_time
print(f"\nOptuna tuning finished in {optuna_time:.1f}s")

[I 2025-04-06 13:32:04,847] A new study created in memory with name: no-name-b9c32933-f481-43d1-a0fd-aad80876c0e2


--- Starting Optuna Hyperparameter Tuning ---
Data shape for Optuna: X=(136409, 19), y=(136409,)
Features used (19): ['year', 'month', 'day', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter', 'is_weekend', 'temp_mean', 'dew_point_mean', 'wind_speed_mean', 'precip_sum', 'user_id', 'daily_kwh_lag_1', 'daily_kwh_lag_2', 'daily_kwh_lag_3', 'daily_kwh_lag_7', 'daily_kwh_lag_14', 'daily_kwh_lag_28']
user_id dtype in X_full_opt: category

Running Optuna study with 20 trials...


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2025-04-06 13:32:46,699] Trial 0 finished with value: 3.1893540520491173 and parameters: {'learning_rate': 0.029360913980860538, 'num_leaves': 24, 'max_depth': 10, 'feature_fraction': 0.7282883809023775, 'bagging_fraction': 0.542364078616247, 'bagging_freq': 2, 'lambda_l1': 9.75148305736885, 'lambda_l2': 0.19982594752523308, 'min_child_samples': 29}. Best is trial 0 with value: 3.1893540520491173.
[I 2025-04-06 13:33:17,955] Trial 1 finished with value: 2.85457616805172 and parameters: {'learning_rate': 0.043793439626086006, 'num_leaves': 33, 'max_depth': 4, 'feature_fraction': 0.5715880746686312, 'bagging_fraction': 0.7992913866343199, 'bagging_freq': 2, 'lambda_l1': 0.35963564290540034, 'lambda_l2': 0.39329506131320896, 'min_child_samples': 21}. Best is trial 1 with value: 2.85457616805172.
[I 2025-04-06 13:33:56,790] Trial 2 finished with value: 2.852914189692552 and parameters: {'learning_rate': 0.08350534311758406, 'num_leaves': 23, 'max_depth': 10, 'feature_fraction': 0.794335

In [18]:
# Temporary Cell (Run this INSTEAD of re-running Cell 9)

print("--- Preparing X_full and y_full for Final Model Training ---")

# --- Check for required inputs ---
if 'df_train_full' not in locals(): raise NameError("df_train_full not found (Run Cell 8).")
if 'features' not in locals(): raise NameError("features list not defined (Run Cell 8).")
if 'target' not in locals(): raise NameError("target variable name not defined (Run Cell 8).")

# --- Prepare dataset (Copied from start of Cell 9) ---
# Ensure data is sorted by date
print("Sorting data by date...")
df_train_full_final = df_train_full.sort_values(by='date') # Use a slightly different name

# Ensure user_id is category type
if df_train_full_final['user_id'].dtype.name != 'category':
    print("Converting user_id to category...")
    df_train_full_final['user_id'] = df_train_full_final['user_id'].astype('category')

# Define X_full and y_full using the feature list from Cell 8
X_full = df_train_full_final[features] # Defines X_full
y_full = df_train_full_final[target]   # Defines y_full

print(f"X_full defined. Shape: {X_full.shape}")
print(f"y_full defined. Shape: {y_full.shape}")
print(f"user_id dtype in X_full: {X_full['user_id'].dtype}")
print("--- Ready to run Cell 10 ---")

--- Preparing X_full and y_full for Final Model Training ---
Sorting data by date...
X_full defined. Shape: (136409, 19)
y_full defined. Shape: (136409,)
user_id dtype in X_full: category
--- Ready to run Cell 10 ---


## 6. Model Training (optuna Baseline)

We will use LightGBM, a gradient boosting framework known for its speed and efficiency, to train a baseline model. We'll use early stopping based on the validation set performance (RMSE) to prevent overfitting and find a reasonable number of boosting rounds.

In [21]:
# Cell 10: Final Model Training (Using Optuna Best Params)

import lightgbm as lgb
import numpy as np
import time

print("\n--- Training Final Model on Full Data using Optuna Params ---")
final_model_start_time = time.time()

# --- Check for required inputs ---
if 'X_full' not in locals(): raise NameError("X_full (features) not found.")
if 'y_full' not in locals(): raise NameError("y_full (target) not found.")
if 'best_params_from_optuna' not in locals(): raise NameError("best_params_from_optuna not found.")
# We still need best_iterations from the *initial* CV run (Cell 9) if we want to use median rounds
# If Cell 9 wasn't re-run after Optuna, best_iterations might be from the old params.
# Safer to train Optuna best params for a fixed large number or estimate rounds differently.
# Let's train for a fixed, reasonably large number of rounds found during tuning.
# Alternatively, refactor to get best rounds from the best Optuna trial if needed.

# --- Initialize Final Model using Optuna Params ---
final_model_params = best_params_from_optuna.copy()
# Add back necessary fixed params if not tuned by Optuna
final_model_params['objective'] = 'regression_l1'
final_model_params['metric'] = 'rmse' # Can be omitted if not monitoring fit
final_model_params['seed'] = SEED
final_model_params['n_jobs'] = -1
final_model_params['verbose'] = -1
final_model_params['boosting_type'] = 'gbdt'
# Set n_estimators - using a fixed high value or estimate from Optuna trials average
# Let's use a fixed value based on previous reasonable results
final_model_params['n_estimators'] = 800 # Adjust based on typical best_iterations seen before
print(f"Using Optuna best params + n_estimators={final_model_params['n_estimators']}")
print(f"Full final parameters: {final_model_params}")

# Ensure user_id is category
if X_full['user_id'].dtype.name != 'category':
     print("Warning: Converting X_full user_id to category for final training.")
     X_full['user_id'] = X_full['user_id'].astype('category')

final_model = lgb.LGBMRegressor(**final_model_params)

# --- Train on ALL data ---
print(f"Training final model on {X_full.shape[0]} samples...")
final_model.fit(X_full, y_full,
                categorical_feature=['user_id'] # Specify categorical feature
               )

final_model_time = time.time() - final_model_start_time
print(f"\nFinal model training complete in {final_model_time:.1f}s")

# Optional: Save the final model
# import joblib
# joblib.dump(final_model, 'final_lgb_model_v7_optuna.pkl')
# print("Final model saved.")


--- Training Final Model on Full Data using Optuna Params ---
Using Optuna best params + n_estimators=800
Full final parameters: {'learning_rate': 0.023638976056168663, 'num_leaves': 31, 'max_depth': 6, 'feature_fraction': 0.606413813222159, 'bagging_fraction': 0.8463930721986744, 'bagging_freq': 6, 'lambda_l1': 0.31294092701733434, 'lambda_l2': 1.1947964494084333, 'min_child_samples': 34, 'objective': 'regression_l1', 'metric': 'rmse', 'seed': 42, 'n_jobs': -1, 'verbose': -1, 'boosting_type': 'gbdt', 'n_estimators': 800}
Training final model on 136409 samples...

Final model training complete in 25.1s


## 7. Local Validation (Calculate RMSE)

Evaluate the trained model's performance on the unseen validation set. This gives us an estimate of how well the model might perform on the actual test data on Zindi. We will calculate the Root Mean Squared Error (RMSE).

## 8. Prepare Test Data and Generate Predictions

Now we prepare the actual test dataset based on the `SampleSubmission.csv` file. This involves:
1.  Extracting the required future `date` and `user_id` from the `ID` column in the sample submission.
2.  Merging the relevant aggregated daily `climate` data for those future dates.
3.  Creating the same date-based `features` that the model was trained on.
4.  Handling any potential missing climate data for future dates (e.g., by forward filling the last known values).
5.  Using the trained `model` to predict `daily_kwh` for the test set.

In [23]:
# Cell 16: Prepare Test Data and Predict (NO Rolling Features, with UserID)

import time
import pandas as pd
import numpy as np

print("--- Preparing Test Data (NO Rolling Features, with UserID) ---")
test_prep_start_time = time.time()

# --- Check for required inputs ---
if 'sample_sub' not in locals(): raise NameError("sample_sub not found.")
if 'daily_climate' not in locals(): raise NameError("daily_climate DataFrame not found.")
if 'df_train_full' not in locals(): raise NameError("df_train_full DataFrame not found.")
if 'create_date_features' not in locals(): raise NameError("create_date_features function not defined.")
# Ensure 'features' list from Cell 8 (NO ROLLING COLS) is used
if 'features' not in locals(): raise NameError("features list not defined.")
if 'final_model' not in locals(): raise NameError("Trained 'final_model' not found.")
# Define lag/rolling lists based on FEATURES list for consistency
lag_cols_created = [f for f in features if 'daily_kwh_lag_' in f]
rolling_cols_created = [f for f in features if 'daily_kwh_roll_' in f] # Should be empty for this run
print(f"Expecting {len(lag_cols_created)} lag features based on 'features' list.")
print(f"Expecting {len(rolling_cols_created)} rolling features based on 'features' list.")


# --- 1. Create Base Test Set Structure ---
test_df_base = sample_sub[['ID']].copy()
try:
    test_df_base['date_str'] = test_df_base['ID'].str.split('_').str[0]
    test_df_base['date'] = pd.to_datetime(test_df_base['date_str'], errors='coerce')
    test_df_base['user_id'] = test_df_base['ID'].str.split('_', n=1).str[1]
    test_df_base.dropna(subset=['date'], inplace=True)
except Exception as e: raise ValueError(f"Error parsing ID: {e}")
print(f"[{time.time() - test_prep_start_time:.1f}s] Base test structure created.")


# --- 2. Merge Future Climate Data ---
print(f"[{time.time() - test_prep_start_time:.1f}s] Merging climate data...")
test_df = pd.merge(test_df_base, daily_climate, on='date', how='left')
test_df.sort_values(by='date', inplace=True)
test_df.ffill(inplace=True)
test_df.bfill(inplace=True)
climate_cols_in_test = [col for col in daily_climate.columns if col != 'date' and col in test_df.columns]
print(f"[{time.time() - test_prep_start_time:.1f}s] Climate data merged.")


# --- 3. Combine History for Feature Calculation ---
print(f"[{time.time() - test_prep_start_time:.1f}s] Combining history...")
# Determine max history needed based ONLY on lags (since rolling is off)
max_lag = max((int(c.split('_')[-1]) for c in lag_cols_created), default=0)
# max_roll_window = max((int(c.split('_')[-2]) for c in rolling_cols_created), default=0) # Not needed now
history_needed_days = max_lag + 1 # Minimal buffer for lags

min_test_date = test_df['date'].min()
cutoff_date = min_test_date - pd.Timedelta(days=history_needed_days)
print(f"Max lag={max_lag}. Need history back to {cutoff_date.date()}")

cols_to_keep_hist = ['user_id', 'date', 'daily_kwh']
historical_data = df_train_full[df_train_full['date'] < min_test_date][cols_to_keep_hist].copy()
test_users = test_df['user_id'].unique()
historical_data = historical_data[historical_data['user_id'].isin(test_users)]
historical_data = historical_data[historical_data['date'] >= cutoff_date]

future_data_cols = ['user_id', 'date'] + climate_cols_in_test
future_data = test_df[future_data_cols].copy()
future_data['daily_kwh'] = np.nan

combined_df = pd.concat([historical_data, future_data], ignore_index=True)
combined_df = combined_df.sort_values(by=['user_id', 'date'])
combined_df.drop_duplicates(subset=['user_id', 'date'], keep='last', inplace=True)
print(f"Combined data shape: {combined_df.shape}")


# --- 4. Calculate Lag Features on Combined Data ---
print(f"[{time.time() - test_prep_start_time:.1f}s] Calculating lag features...")
if lag_cols_created:
    target_col_for_lags = 'daily_kwh'
    lags_to_create = [int(c.split('_')[-1]) for c in lag_cols_created]
    if combined_df['user_id'].dtype.name != 'category':
        print("Converting combined_df user_id to category...")
        combined_df['user_id'] = combined_df['user_id'].astype('category')
    for lag in lags_to_create:
        col_name = f"daily_kwh_lag_{lag}"
        combined_df[col_name] = combined_df.groupby('user_id', observed=True)[target_col_for_lags].shift(lag)
    print("Lag calculation finished.")


# --- 5. Calculate Rolling Window Features (SKIPPED) ---
print(f"\n[{time.time() - test_prep_start_time:.1f}s] Skipping rolling window feature calculation.")
# (Keep rolling feature calculation code commented out or deleted)


# --- 6. Create Date Features ---
print(f"\n[{time.time() - test_prep_start_time:.1f}s] Creating date features...")
combined_df = create_date_features(combined_df, 'date')
print(f"Date features created.")


# --- 7. Filter Back to Required Test Dates ---
print(f"\n[{time.time() - test_prep_start_time:.1f}s] Filtering back to required test dates...")
# Use the original test_df_base which has the 'ID' column
test_df_final = pd.merge(test_df_base[['ID', 'user_id', 'date']], combined_df, on=['user_id', 'date'], how='left') #<<< test_df_final DEFINED HERE
print(f"Filtered back to test set shape: {test_df_final.shape}")
if len(test_df_final) != len(sample_sub): print(f"WARNING: Row count mismatch! Got {len(test_df_final)}")


# --- 8. Final Checks and Feature Alignment ---
print(f"\n[{time.time() - test_prep_start_time:.1f}s] Final checks and feature alignment...")

# Convert user_id to category in test_df_final
if 'user_id' in test_df_final.columns:
    if test_df_final['user_id'].dtype.name != 'category':
        print("Converting user_id in test_df_final to category dtype...")
        test_df_final['user_id'] = test_df_final['user_id'].astype('category')
else: print("Warning: user_id column not found in test_df_final.")

# Check feature availability using the 'features' list from Cell 8 (should NOT contain rolling)
features_in_test = [f for f in features if f in test_df_final.columns]
missing_features_in_test = [f for f in features if f not in test_df_final.columns]
if missing_features_in_test:
    print(f"--- ERROR: Features defined in training missing from final test df: {missing_features_in_test} ---")
    print(f"Columns available: {test_df_final.columns.tolist()}")
    raise ValueError("Mismatch between training 'features' list and test_df columns.")

# Handle NaNs (excluding categorical user_id)
print(f"Handling NaNs in final test features (filling with 0)...")
numeric_features_in_test = [f for f in features_in_test if f != 'user_id']
test_df_final[numeric_features_in_test] = test_df_final[numeric_features_in_test].fillna(0)

# Define X_test using ONLY the features available AND used in training
X_test = test_df_final[features_in_test] # <<< X_test DEFINED HERE
if X_test.isnull().any().any():
    print("--- FATAL ERROR: NaNs found in X_test before prediction! ---")
    raise ValueError("NaNs detected in X_test.")
else:
    print("[ OK ] Final test features (X_test) prepared. Shape: {X_test.shape}")


# --- 9. Predict on Test Set using FINAL MODEL ---
print(f"\n[{time.time() - test_prep_start_time:.1f}s] Predicting using final_model...")
model_features = final_model.feature_name_ # Features model was trained on
predict_features = [f for f in model_features if f in X_test.columns] # Ensure order and availability

if len(predict_features) != len(model_features):
    print(f"Warning: Feature mismatch! Model trained on {len(model_features)}, predicting on {len(predict_features)}")
    X_test_reordered = X_test[predict_features]
else:
    X_test_reordered = X_test[model_features] # Use model's feature order

test_predictions = final_model.predict(X_test_reordered)
test_predictions_non_negative = np.maximum(0, test_predictions) # <<< test_predictions_non_negative DEFINED HERE
if np.sum(test_predictions < 0) > 0: print("Note: Negative predictions clipped.")

print("Prediction on test set complete.")
test_prep_total_time = time.time() - test_prep_start_time
print(f"\n[{test_prep_total_time:.1f}s] Cell 16 processing finished.")

--- Preparing Test Data (NO Rolling Features, with UserID) ---
Expecting 6 lag features based on 'features' list.
Expecting 0 rolling features based on 'features' list.
[0.0s] Base test structure created.
[0.0s] Merging climate data...
[0.1s] Climate data merged.
[0.1s] Combining history...
Max lag=28. Need history back to 2024-08-26
Combined data shape: (11640, 7)
[0.2s] Calculating lag features...
Converting combined_df user_id to category...
Lag calculation finished.

[0.2s] Skipping rolling window feature calculation.

[0.2s] Creating date features...
Date features created.

[0.2s] Filtering back to required test dates...
Filtered back to test set shape: (6014, 22)

[0.2s] Final checks and feature alignment...
Converting user_id in test_df_final to category dtype...
Handling NaNs in final test features (filling with 0)...
[ OK ] Final test features (X_test) prepared. Shape: {X_test.shape}

[0.2s] Predicting using final_model...
Note: Negative predictions clipped.
Prediction on test

 Generate submission file

In [24]:
# Cell 18: Generate Submission File (v7 - Optuna Tuned)

print("Generating submission file...")

submission_df = pd.DataFrame({'ID': test_df_final['ID'], 'kwh': test_predictions_non_negative})

# --- CHANGED FILENAME ---
submission_filename = 'submission_v7_optuna_tuned.csv'

submission_df.to_csv(submission_filename, index=False)

print("\n--- Submission File Head ---")
print(submission_df.head())
print(f"\nSubmission file saved successfully as: {submission_filename}")
# (Keep row count check)

Generating submission file...

--- Submission File Head ---
                                          ID       kwh
0  2024-09-24_consumer_device_12_data_user_1  0.158338
1  2024-09-25_consumer_device_12_data_user_1  0.076580
2  2024-09-26_consumer_device_12_data_user_1  0.026149
3  2024-09-27_consumer_device_12_data_user_1  0.017782
4  2024-09-28_consumer_device_12_data_user_1  0.013799

Submission file saved successfully as: submission_v7_optuna_tuned.csv


--- End of Baseline Notebook ---

Next steps:
- Submit the generated CSV to Zindi.
- Analyze the results (local RMSE vs Zindi score).
- Improve the model by:
    - Adding more features (lags, rolling windows, user features).
    - Tuning hyperparameters (e.g., using Optuna).
    - Trying different models (XGBoost, CatBoost).
    - Implementing more robust validation (Time Series Cross-Validation).
    - Ensembling models.